In [14]:
import pickle
import sqlalchemy

from time import time
from tqdm import tqdm
from sqlalchemy import Table, Column, Integer, String, ForeignKey, Binary

In [58]:
def connect(user, password, db, host='localhost', port=5432):
    '''Returns a connection and a metadata object'''
    
    url = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)

    try:
        con = sqlalchemy.create_engine(url, client_encoding='utf8')
        meta = sqlalchemy.MetaData(bind=con, reflect=True)
    except Exception as e:
        if "does not exist" in str(e):
            url = 'postgresql://{}:{}@{}:{}/template1'
            url = url.format(user, password, host, port)
            con = sqlalchemy.create_engine(url, client_encoding='utf8')
            meta = sqlalchemy.MetaData(bind=con, reflect=True)
        else:
            raise e
        
    return con, meta


con, meta = connect('postgres', 'mysecretpassword', 'jobs')
con.execute("""DROP TABLE IF EXISTS jobs""")

/home/needshelp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  if __name__ == '__main__':
/home/needshelp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  from ipykernel import kernelapp as app


In [59]:
size = 10000
JOBS = Table(
         f"jobs", meta,
         Column("id", Integer, primary_key=True),
         Column("status", String),
         Column("name", String),
         Column("subjobs", Integer),
         Column("application", String),
         Column("backend", String),
         Column("backend_actualCE", String),
         Column("comment", String),
         extend_existing=True
    )

BLOBS = Table(
        f"blobs", meta,
        Column("jid", Integer, ForeignKey("JOBS.id")),
        Column("inputsandbox", Binary),
        Column("outputsandbox", Binary),
        Column("info", Binary),
        Column("comment", Binary),
        Column("time", Binary),
        Column("application", Binary),
        Column("backend", Binary),
        Column("inputfiles", Binary),
        Column("outputfiles", Binary),
        Column("non_copyable_outputfiles", Binary),
        Column("id", Binary),
        Column("status", Binary),
        Column("name", Binary),
        Column("inputdir", Binary),
        Column("outputdir", Binary),
        Column("inputdata", Binary),
        Column("outputdata", Binary),
        Column("splitter", Binary),
        Column("subjobs", Binary),
        Column("master", Binary),
        Column("postprocessors", Binary),
        Column("virtualization", Binary),
        Column("merger", Binary),
        Column("do_auto_resubmit", Binary),
        Column("metadata", Binary),
        Column("been_queued", Binary),
        Column("parallel_submit", Binary),
        extend_existing=True
    )

meta.create_all(con)

In [61]:
jobs = pickle.load(open("rows.pkl", "rb"))
blobs = pickle.load(open("blobs.pkl", "rb"))

In [54]:
x

sqlalchemy.exc.IntegrityError('(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "JOBS_pkey"\nDETAIL:  Key (id)=(1101) already exists.\n')

In [62]:
size = 100000
_jobs = jobs[:size]
_blobs = blobs[:size]
with tqdm(total=size) as progress:
    for i, (row, blob) in enumerate(zip(_jobs, _blobs)):
        row[0] = i+1
        blob['jid'] = i+1
        insert_job = JOBS.insert().values(row)
        try:
            con.execute(insert_job)      
        except Exception as e:
            x = e
            pass
        progress.update(1)

100%|██████████| 100000/100000 [05:08<00:00, 324.02it/s]


In [40]:
from joblib import Parallel, delayed
rows = [[i]+row[1:] for i, row in enumerate(_jobs)]
def insert_postgres(data):
    insert_job = JOBS.insert().values(row)
    flag = con.execute(insert_job)      
    return flag

In [63]:
%%time
rows = con.execute("SELECT * FROM JOBS")

CPU times: user 30.4 ms, sys: 13.1 ms, total: 43.5 ms
Wall time: 141 ms


In [66]:
%%time
len(list(rows))

CPU times: user 23 µs, sys: 3 µs, total: 26 µs
Wall time: 30.8 µs


0